<a href="https://colab.research.google.com/github/gdmaru/GA_DAT5/blob/master/LEGO_Classification_via_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LEGO Project

I want to classify legos 

I'd like to use the famous imagenet model called Rasnet50 to do a transfer learning

## Testing for GPU 


To Test if you have GPU set up

Run the Cell below

if no GPU is found press Runtime (in the menu at the top) and choose "Change Runtime Type" to GPU


In [0]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## Google Drive Setup

In [0]:
from google.colab import auth
auth.authenticate_user()

In [0]:
from google.colab import drive
drive.mount('/gdrive')

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [0]:
!ls '/gdrive/My Drive/'

 basic_cnn.h5
'Colab Notebooks'
'Dongchan Lee - CV - 05 Jan 2018.doc'
 git_and_project_submissions.gslides
'git_and_project_submissions의 사본.gslides'
'Global Superstore (1).xls.gsheet'
'Global Superstore (2).xls.gsheet'
'Global Superstore (3).xls.gsheet'
'Global Superstore.xls.gsheet'
 LEGO.zip
 mnist_cnn_model.h5
'test01 (1).txt'
 test01.txt


##Unzip the data
by placing ! infront we can run almost any linux shell command

In [0]:
!unzip -qq '/gdrive/My Drive/LEGO.zip'

!ls

replace LEGO/306201/IMG_9621.JPG? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
adc.json  data	LEGO  __MACOSX	sample_data


In [0]:
! rm LEGO/*/.DS*

In [0]:
!ls -la LEGO/6174310

total 7404
drwxr-xr-x  2 root root    4096 Dec 14 09:42 .
drwxr-xr-x 33 root root    4096 Dec 14 09:42 ..
-rw-r--r--  1 root root 1408406 Dec 14 06:08 IMG_9643.JPG
-rw-r--r--  1 root root 1666918 Dec 14 06:08 IMG_9644.JPG
-rw-r--r--  1 root root 1708854 Dec 14 06:08 IMG_9645.JPG
-rw-r--r--  1 root root 1449265 Dec 14 06:08 IMG_9646.JPG
-rw-r--r--  1 root root 1332355 Dec 14 06:08 IMG_9647.JPG


In [0]:
CLASS_DIR='LEGO'
classes = [d for d in os.listdir(CLASS_DIR) if not d.startswith('.')]
', '.join(classes)

'6108662, 300423, 300523, 449521, 246001, 614323, 428623, 4275815, 300123, 6174310, 6099548, 303221, 6079402, 306201, 4173943, 300121, 6099412, 6054551, 6069002, 300421, 302401, 6030266, 366623, 6060734, 371021, 6218943, 4534648, 302023, 6072998, 9342, 303923'

In [0]:
#from tensorflow.python.keras.applications.nasnet import NASNetLarge, preprocess_input
#model = NASNetLarge(weights='imagenet', include_top=False)  # 343,608,736

#from tensorflow.keras.applications.nasnet import NASNetMobile, preprocess_input, decode_predictions
from keras.applications.nasnet import NASNetMobile, preprocess_input, decode_predictions

model_imagenet = NASNetMobile(weights='imagenet', include_top=True)   # 24,226,656 bytes
print("Model Loaded")

24231936/24227760 [==============================] - 3s 0us/step
Model Loaded


In [0]:
# https://www.tensorflow.org/api_docs/python/tf/keras/applications/
#from tensorflow.keras.preprocessing import image as keras_preprocessing_image
from keras.preprocessing import image as keras_preprocessing_image

In [0]:
#model_logits   = NASNetMobile(weights='imagenet', include_top=False, pooling=None)  # 19,993,200 bytes
#logits_layer = model_imagenet.get_layer('global_average_pooling2d_1')
logits_layer = model_imagenet.get_layer('predictions')
model_logits = keras.Model(inputs=model_imagenet.input, outputs=logits_layer.output)
print("Model Loaded")

Model Loaded


In [0]:
def crop_middle_square_area(np_image):
    h, w, _ = np_image.shape
    h = int(h/2)
    w = int(w/2)
    if h>w:
        return np_image[ h-w:h+w, : ]
    return np_image[ :, w-h:w+h ]    
  
#im_sq = crop_middle_square_area(im)
#im_sq.shape

In [0]:
def image_to_input(model, img_path):
    target_size=model.input_shape[1:]
    img = keras_preprocessing_image.load_img(img_path, target_size=target_size)
    
    x = keras_preprocessing_image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)

    return x

In [0]:
train = dict(filepath=[], features=[], target=[])

t0 = time.time()

for class_i, directory in enumerate(classes):
    for filename in os.listdir(os.path.join(CLASS_DIR, directory)):
        filepath = os.path.join(CLASS_DIR, directory, filename)
        if os.path.isdir(filepath): continue

        im = plt.imread(filepath)
        im_sq = crop_middle_square_area(im)

        x = image_to_input(model_logits, filepath)
        #np_logits = model_logits.predict(x)  # Shape = 1x7x7x1056  if pooling=None
        #print(np_logits.shape)
        #np_logits_pooled = get_logits_from_non_top( np_logits )
        
        np_logits_pooled = model_logits.predict(x)[0]  # Shape = 1x1056 if pooling=avg
        
        train['filepath'].append(filepath)
        train['features'].append(np_logits_pooled)
        train['target'].append( class_i )

        #plt.figure()
        #plt.imshow(im_sq.astype('uint8'))
        #plt.axis('off')

        #plt.text(2*320, 50, '{}'.format(filename), fontsize=14)
        #plt.text(2*320, 80, 'Train as class "{}"'.format(directory), fontsize=12)

print("DONE : %6.2f seconds each" %(float(time.time() - t0)/len(train),))

DONE :  44.12 seconds each


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt


import os, json
from glob import glob
import numpy as np

import keras
import time

from tensorflow.keras import optimizers

from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.applications.vgg16 import VGG16


from tensorflow.python.keras.preprocessing import image
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

from tensorflow.python.keras.models import Model,load_model,Sequential

from tensorflow.python.keras.layers import Dense, GlobalAveragePooling2D, Dropout,Flatten, Input, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.python.keras import backend as K

from tensorflow.keras.callbacks import ModelCheckpoint, Callback


In [0]:
print(tf.__version__)
print(tf.keras.__version__)
print(keras.__version__)

1.12.0
2.1.6-tf
2.2.4


In [0]:
# Set up some Hyperparameters

batch_size = 32
epochs = 10

lr = 0.001